# Description

Generates manubot tables for PhenomeXcan and eMERGE associations given an LV name (which is the only parameter that needs to be specified in the Settings section below).

# Modules loading

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import re
from pathlib import Path

import pandas as pd

from entity import Trait
import conf

# Settings

In [3]:
LV_NAME = "LV5"

In [4]:
assert (
    conf.MANUSCRIPT["BASE_DIR"] is not None
), "The manuscript directory was not configured"

OUTPUT_FILE_PATH = conf.MANUSCRIPT["CONTENT_DIR"] / "50.00.supplementary_material.md"
display(OUTPUT_FILE_PATH)
assert OUTPUT_FILE_PATH.exists()

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier_manuscript/content/50.00.supplementary_material.md')

In [5]:
# result_set is either phenomexcan or emerge
LV_FILE_MARK_TEMPLATE = "<!-- {lv}:{result_set}_traits_assocs:{position} -->"

In [6]:
TABLE_CAPTION = "Table: Significant trait associations of {lv_name} in {result_set_name}. {table_id}"

In [7]:
TABLE_CAPTION_ID = "#tbl:sup:{result_set}_assocs:{lv_name_lower_case}"

In [8]:
RESULT_SET_NAMES = {
    "phenomexcan": "PhenomeXcan",
    "emerge": "eMERGE",
}

# Load data

## PhenomeXcan LV-trait associations

In [9]:
input_filepath = Path(conf.RESULTS["GLS"] / "gls_phenotypes-combined-phenomexcan.pkl")
display(input_filepath)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/bases_data/base_orig/results/gls/gls_phenotypes-combined-phenomexcan.pkl')

In [10]:
phenomexcan_lv_trait_assocs = pd.read_pickle(input_filepath)

In [11]:
phenomexcan_lv_trait_assocs.shape

(5782, 8)

In [12]:
phenomexcan_lv_trait_assocs.head()

,part_k,cluster_id,phenotype,lv,coef,pvalue,pvalue_twosided,fdr
0,29,22,100002_raw-Energy,LV246,0.002401,0.424213,0.848425,0.680765
1,29,22,100002_raw-Energy,LV607,-0.006323,0.691499,0.617002,0.873350
2,29,22,100002_raw-Energy,LV612,-0.000822,0.525786,0.948428,0.763843
3,29,22,100002_raw-Energy,LV74,-0.006035,0.685313,0.629373,0.870769
4,29,22,100002_raw-Energy,LV838,0.024454,0.023446,0.046891,0.087403


## eMERGE LV-trait associations

In [13]:
input_filepath = Path(conf.RESULTS["GLS"] / "gls_phenotypes-combined-emerge.pkl")
display(input_filepath)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/bases_data/base_orig/results/gls/gls_phenotypes-combined-emerge.pkl')

In [14]:
emerge_lv_trait_assocs = pd.read_pickle(input_filepath)

In [15]:
emerge_lv_trait_assocs.shape

(7725, 6)

In [16]:
emerge_lv_trait_assocs.head()

,phenotype,lv,coef,pvalue,pvalue_twosided,fdr
0,008,LV246,-0.011008,0.809325,0.381350,0.999418
1,008,LV30,0.022387,0.038699,0.077397,0.483734
2,008,LV57,0.003960,0.379121,0.758243,0.961449
3,008,LV865,-0.008742,0.758337,0.483327,0.999418
4,008,LV847,-0.028246,0.985827,0.028346,0.999418


## eMERGE traits info

In [17]:
input_filepath = conf.EMERGE["DESC_FILE_WITH_SAMPLE_SIZE"]
display(input_filepath)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/bases_data/base_orig/data/emerge/eMERGE_III_PMBB_GSA_v2_2020_phecode_AFR_EUR_cc50_counts_w_dictionary.txt')

In [18]:
emerge_traits_info = pd.read_csv(
    input_filepath,
    sep="\t",
    dtype={"phecode": str},
    usecols=[
        "phecode",
        "phenotype",
        "category",
        "eMERGE_III_EUR_case",
        "eMERGE_III_EUR_control",
    ],
)

In [19]:
emerge_traits_info = emerge_traits_info.set_index("phecode")

In [20]:
emerge_traits_info = emerge_traits_info.rename(
    columns={
        "eMERGE_III_EUR_case": "eur_n_cases",
        "eMERGE_III_EUR_control": "eur_n_controls",
    }
)

In [21]:
emerge_traits_info.shape

(309, 4)

In [22]:
emerge_traits_info.head()

,eur_n_cases,eur_n_controls,phenotype,category
phecode,,,,
008,1639,57495,Intestinal infection,infectious diseases
008.5,1024,57495,Bacterial enteritis,infectious diseases
008.52,893,57495,Intestinal infection due to C. difficile,infectious diseases
038,3172,50610,Septicemia,infectious diseases
038.3,1361,50610,Bacteremia,infectious diseases


In [23]:
assert emerge_traits_info.index.is_unique

# Trait associations

## PhenomeXcan

In [24]:
from traits import SHORT_TRAIT_NAMES

In [25]:
result_set = "phenomexcan"

In [26]:
def get_trait_objs(phenotype_full_code):
    if Trait.is_efo_label(phenotype_full_code):
        traits = Trait.get_traits_from_efo(phenotype_full_code)
    else:
        traits = [Trait.get_trait(full_code=phenotype_full_code)]

    # sort by sample size
    return sorted(traits, key=lambda x: x.n_cases / x.n, reverse=True)


def get_trait_description(phenotype_full_code):
    traits = get_trait_objs(phenotype_full_code)

    desc = traits[0].description
    if desc in SHORT_TRAIT_NAMES:
        return SHORT_TRAIT_NAMES[desc]

    return desc


def get_trait_n(phenotype_full_code):
    traits = get_trait_objs(phenotype_full_code)

    return traits[0].n


def get_trait_n_cases(phenotype_full_code):
    traits = get_trait_objs(phenotype_full_code)

    return traits[0].n_cases


def num_to_int_str(num):
    if pd.isnull(num):
        return ""

    return f"{num:,.0f}"


def get_part_clust(row):
    return f"{row.part_k} / {row.cluster_id}"

In [27]:
lv_assocs = phenomexcan_lv_trait_assocs[
    (phenomexcan_lv_trait_assocs["lv"] == LV_NAME)
    & (phenomexcan_lv_trait_assocs["fdr"] < 0.05)
].sort_values("fdr")

In [28]:
with pd.option_context(
    "display.max_rows", None, "display.max_columns", None, "display.max_colwidth", None
):
    display(lv_assocs)

,part_k,cluster_id,phenotype,lv,coef,pvalue,pvalue_twosided,fdr
4770,29,16,MAGNETIC_LDL.C,LV5,0.054256,0.000011,0.000023,0.000144
4762,29,16,MAGNETIC_IDL.TG,LV5,0.051101,0.000038,0.000076,0.000403
4831,29,16,alzheimer's disease,LV5,0.043160,0.000317,0.000634,0.002474
2979,29,27,20446-Ever_had_prolonged_feelings_of_sadness_or_depression,LV5,0.039473,0.001063,0.002125,0.006919
3656,29,27,20546_3-Substances_taken_for_depression_Medication_prescribed_to_you_for_at_least_two_weeks,LV5,0.038465,0.001305,0.002610,0.008236
3418,29,27,20510-Recent_feelings_of_depression,LV5,0.037390,0.001759,0.003518,0.010496
3097,29,27,20485-Ever_contemplated_selfharm,LV5,0.035794,0.002716,0.005431,0.015011
3497,29,27,20514-Recent_lack_of_interest_or_pleasure_in_doing_things,LV5,0.035283,0.003061,0.006123,0.016543
2877,29,27,20403-Amount_of_alcohol_drunk_on_a_typical_drinking_day,LV5,0.032401,0.005760,0.011520,0.027684
3259,29,27,20499-Ever_sought_or_received_professional_help_for_mental_distress,LV5,0.031715,0.006605,0.013210,0.030849


In [29]:
lv_assocs = lv_assocs.assign(
    phenotype_desc=lv_assocs["phenotype"].apply(get_trait_description)
)

In [30]:
lv_assocs = lv_assocs.assign(n=lv_assocs["phenotype"].apply(get_trait_n))

In [31]:
lv_assocs = lv_assocs.assign(n_cases=lv_assocs["phenotype"].apply(get_trait_n_cases))

In [32]:
lv_assocs = lv_assocs.assign(coef=lv_assocs["coef"].apply(lambda x: f"{x:.3f}"))

In [33]:
lv_assocs = lv_assocs.assign(
    fdr=lv_assocs["fdr"].apply(lambda x: f"{x:.2e}".replace("-", "&#8209;"))
)

In [34]:
lv_assocs = lv_assocs.assign(n=lv_assocs["n"].apply(num_to_int_str))

In [35]:
lv_assocs = lv_assocs.assign(n_cases=lv_assocs["n_cases"].apply(num_to_int_str))

In [36]:
lv_assocs = lv_assocs.assign(part_clust=lv_assocs.apply(get_part_clust, axis=1))

In [37]:
lv_assocs = lv_assocs.drop(columns=["phenotype"])

In [38]:
lv_assocs.shape

(12, 11)

In [39]:
lv_assocs = lv_assocs[["phenotype_desc", "n", "n_cases", "part_clust", "fdr"]]

In [40]:
lv_assocs = lv_assocs.rename(
    columns={
        "part_clust": "Partition / cluster",
        "lv": "Latent variable (LV)",
        #         "coef": r"$\beta$",
        "fdr": "FDR",
        "phenotype_desc": "Trait description",
        "n": "Sample size",
        "n_cases": "Cases",
    }
)

In [41]:
with pd.option_context(
    "display.max_rows", None, "display.max_columns", None, "display.max_colwidth", None
):
    display(lv_assocs)

,Trait description,Sample size,Cases,Partition / cluster,FDR
4770,LDL Cholesterol NMR,"13,527",,29 / 16,1.44e&#8209;04
4762,Triglycerides NMR,"21,559",,29 / 16,4.03e&#8209;04
4831,Alzheimers Disease,"54,162","17,008",29 / 16,2.47e&#8209;03
2979,Ever had prolonged feelings of sadness or depression,"117,763","64,374",29 / 27,6.92e&#8209;03
3656,Medication for depression,"117,763","28,351",29 / 27,8.24e&#8209;03
3418,Recent feelings of depression,"117,656",,29 / 27,1.05e&#8209;02
3097,Ever contemplated self-harm,"117,610",,29 / 27,1.50e&#8209;02
3497,Recent lack of interest or pleasure in doing things,"117,757",,29 / 27,1.65e&#8209;02
2877,Amount of alcohol drunk on a typical drinking day,"108,256",,29 / 27,2.77e&#8209;02
3259,Ever sought or received professional help for mental distress,"117,677","46,020",29 / 27,3.08e&#8209;02


### Fill empty

In [42]:
if lv_assocs.shape[0] == 0:
    lv_assocs.loc[0, "Trait description"] = "No significant associations"
    lv_assocs = lv_assocs.fillna("")

### Save

In [43]:
# start
lv_file_mark_start = LV_FILE_MARK_TEMPLATE.format(
    result_set=result_set, lv=LV_NAME, position="start"
)
display(lv_file_mark_start)

# end
lv_file_mark_end = LV_FILE_MARK_TEMPLATE.format(
    result_set=result_set, lv=LV_NAME, position="end"
)
display(lv_file_mark_end)

'<!-- LV5:phenomexcan_traits_assocs:start -->'

'<!-- LV5:phenomexcan_traits_assocs:end -->'

In [44]:
new_content = lv_assocs.to_markdown(index=False, disable_numparse=True)

In [45]:
# add table caption
table_caption = TABLE_CAPTION.format(
    lv_name=LV_NAME,
    result_set_name=RESULT_SET_NAMES[result_set],
    table_id="{"
    + TABLE_CAPTION_ID.format(result_set=result_set, lv_name_lower_case=LV_NAME.lower())
    + "}",
)
display(table_caption)

'Table: Significant trait associations of LV5 in PhenomeXcan. {#tbl:sup:phenomexcan_assocs:lv5}'

In [46]:
new_content += "\n\n" + table_caption

In [47]:
full_new_content = (
    lv_file_mark_start + "\n" + new_content.strip() + "\n" + lv_file_mark_end
)

In [48]:
with open(OUTPUT_FILE_PATH, "r", encoding="utf8") as f:
    file_content = f.read()

In [49]:
new_file_content = re.sub(
    lv_file_mark_start + ".*?" + lv_file_mark_end,
    full_new_content,
    file_content,
    flags=re.DOTALL,
)

In [50]:
with open(OUTPUT_FILE_PATH, "w", encoding="utf8") as f:
    f.write(new_file_content)  # .replace("\beta", r"\beta"))

## eMERGE

In [51]:
result_set = "emerge"

In [52]:
TABLE_CAPTION = (
    "Table: Trait associations of {lv_name} in {result_set_name}. {table_id}"
)

In [53]:
lv_assocs = emerge_lv_trait_assocs[
    (emerge_lv_trait_assocs["lv"] == LV_NAME) & (emerge_lv_trait_assocs["fdr"] < 0.10)
].sort_values("fdr")

In [54]:
with pd.option_context(
    "display.max_rows", None, "display.max_columns", None, "display.max_colwidth", None
):
    display(lv_assocs)

,phenotype,lv,coef,pvalue,pvalue_twosided,fdr
653,241,LV5,0.046211,0.000152,0.000303,0.019188
677,241.1,LV5,0.039263,0.001062,0.002124,0.068359


In [55]:
lv_assocs = lv_assocs.assign(
    phenotype_desc=lv_assocs["phenotype"].apply(
        lambda x: emerge_traits_info.loc[x, "phenotype"]
    )
)

In [56]:
lv_assocs = lv_assocs.assign(
    n=lv_assocs["phenotype"].apply(
        lambda x: emerge_traits_info.loc[x, ["eur_n_cases", "eur_n_controls"]].sum()
    )
)

In [57]:
lv_assocs = lv_assocs.assign(
    n_cases=lv_assocs["phenotype"].apply(
        lambda x: emerge_traits_info.loc[x, "eur_n_cases"]
    )
)

In [58]:
lv_assocs = lv_assocs.assign(coef=lv_assocs["coef"].apply(lambda x: f"{x:.3f}"))

In [59]:
lv_assocs = lv_assocs.assign(
    fdr=lv_assocs["fdr"].apply(lambda x: f"{x:.2e}".replace("-", "&#8209;"))
)

In [60]:
lv_assocs = lv_assocs.assign(n=lv_assocs["n"].apply(num_to_int_str))

In [61]:
lv_assocs = lv_assocs.assign(n_cases=lv_assocs["n_cases"].apply(num_to_int_str))

In [62]:
lv_assocs = lv_assocs.rename(columns={"phenotype": "phecode"})

In [63]:
lv_assocs.shape

(2, 9)

In [64]:
lv_assocs = lv_assocs[["phecode", "phenotype_desc", "n", "n_cases", "fdr"]]

In [65]:
lv_assocs = lv_assocs.rename(
    columns={
        "lv": "Latent variable (LV)",
        #         "coef": r"$\beta$",
        "fdr": "FDR",
        "phecode": "Phecode",
        "phenotype_desc": "Trait description",
        "n": "Sample size",
        "n_cases": "Cases",
    }
)

In [66]:
with pd.option_context(
    "display.max_rows", None, "display.max_columns", None, "display.max_colwidth", None
):
    display(lv_assocs)

,Phecode,Trait description,Sample size,Cases,FDR
653,241,Nontoxic nodular goiter,"47,842","3,158",1.92e&#8209;02
677,241.1,Nontoxic uninodular goiter,"47,125","2,441",6.84e&#8209;02


### Fill empty

In [67]:
if lv_assocs.shape[0] == 0:
    lv_assocs.loc[0, "Phecode"] = "No significant associations"
    lv_assocs = lv_assocs.fillna("")

### Save

In [68]:
# start
lv_file_mark_start = LV_FILE_MARK_TEMPLATE.format(
    result_set=result_set, lv=LV_NAME, position="start"
)
display(lv_file_mark_start)

# end
lv_file_mark_end = LV_FILE_MARK_TEMPLATE.format(
    result_set=result_set, lv=LV_NAME, position="end"
)
display(lv_file_mark_end)

'<!-- LV5:emerge_traits_assocs:start -->'

'<!-- LV5:emerge_traits_assocs:end -->'

In [69]:
new_content = lv_assocs.to_markdown(index=False, disable_numparse=True)

In [70]:
# add table caption
table_caption = TABLE_CAPTION.format(
    lv_name=LV_NAME,
    result_set_name=RESULT_SET_NAMES[result_set],
    table_id="{"
    + TABLE_CAPTION_ID.format(result_set=result_set, lv_name_lower_case=LV_NAME.lower())
    + "}",
)
display(table_caption)

'Table: Trait associations of LV5 in eMERGE. {#tbl:sup:emerge_assocs:lv5}'

In [71]:
new_content += "\n\n" + table_caption

In [72]:
full_new_content = (
    lv_file_mark_start + "\n" + new_content.strip() + "\n" + lv_file_mark_end
)

In [73]:
with open(OUTPUT_FILE_PATH, "r", encoding="utf8") as f:
    file_content = f.read()

In [74]:
new_file_content = re.sub(
    lv_file_mark_start + ".*?" + lv_file_mark_end,
    full_new_content,
    file_content,
    flags=re.DOTALL,
)

In [75]:
with open(OUTPUT_FILE_PATH, "w", encoding="utf8") as f:
    f.write(new_file_content)  # .replace("\beta", r"\beta"))